In [1]:
import oda_api.token 
import logging
import numpy as np
from oda_api.api import DispatcherAPI
from oda_api.plot_tools import OdaImage, OdaLightCurve, OdaSpectrum
import matplotlib.pyplot as plt
import astroquery.heasarc
from astropy.wcs import WCS
from astropy.io import fits
from astroquery.simbad import Simbad
from astropy import coordinates as coord
from astropy.coordinates import SkyCoord
from astropy.time import Time
from matplotlib.patches import Circle
from astroquery.jplhorizons import Horizons
import pandas as pd
import astropy.units as u
import json
from collections import defaultdict

In [2]:
logging.getLogger().setLevel(logging.WARNING)
logging.getLogger('oda_api').addHandler(logging.StreamHandler())

Load the ScWs

In [3]:
scw_ids = []
scw_versions = []
scw_start_times = []
scw_end_times = []
jupiter_ra = []
jupiter_dec = []

with open("../data/2003-01-01_2025-01-01.txt", "r") as f:
    next(f)
    for line in f:
        parts = line.strip().split(", ")
        scw_ids.append(parts[0])
        scw_versions.append(parts[1])
        scw_start_times.append(float(parts[2]))  
        scw_end_times.append(float(parts[3]))  
        jupiter_ra.append(float(parts[4]))  
        jupiter_dec.append(float(parts[5]))  

unique_sorted_data = {}
for sid, ver, start, end, ra, dec in sorted(zip(scw_ids, scw_versions, scw_start_times, scw_end_times, jupiter_ra, jupiter_dec), key=lambda x: x[0]):
    if sid not in unique_sorted_data:  
        unique_sorted_data[sid] = (sid, ver, start, end, ra, dec)

scw_ids, scw_versions, start, end, ra, dec = map(list, zip(*unique_sorted_data.values()))

scw = [id + "." + ver for id, ver in zip(scw_ids, scw_versions)]
durations = [(e - s) for e,s in zip(end, start)]

## QUERYING 

## IMAGES

In [4]:
isot_start_times = Time(start, format='mjd').isot
isot_end_times = Time(end, format='mjd').isot
duration_seconds = [duration * 86400 for duration in durations]
year_months = [st[:7] for st in isot_start_times]

How many ScWs after filtering for pointings?

In [ ]:
scw_count_by_year_month = {}

for year_month in sorted(set(year_months)):
    filtered_scws = [
        (scw[i], isot_start_times[i], isot_end_times[i], jupiter_ra[i], jupiter_dec[i])
        for i, year_month_in_list in enumerate(year_months)
        if year_month_in_list == year_month
    ]

    if not filtered_scws:
        scw_count_by_year_month[year_month] = 0
        continue

    rrrr_max_pppp = defaultdict(int)
    for scwgr in filtered_scws:
        rrrr = scwgr[0][:4]  
        pppp = int(scwgr[0][4:8])
        rrrr_max_pppp[rrrr] = max(rrrr_max_pppp[rrrr], pppp)

    filtered_scws = [
        scwind for scwind in filtered_scws
        if 5 < int(scwind[0][4:8]) < rrrr_max_pppp[scwind[0][:4]] - 10
    ]

    scw_count_by_year_month[year_month] = len(filtered_scws)

print(f"Total SCWs after filtering: {sum(scw_count_by_year_month.values())}")


Total SCWs after filtering: 47


In [ ]:
disp_by_date = {}
data_by_date = {}
successful_scws = []

disp = DispatcherAPI(url="https://www.astro.unige.ch/mmoda/dispatch-data", instrument="mock")

while True:
    image_results = []

    for year_month in sorted(set(year_months)):
        filtered_scws = [
            (scw[i], isot_start_times[i], isot_end_times[i], jupiter_ra[i], jupiter_dec[i])
            for i, year_month_in_list in enumerate(year_months)
            if year_month_in_list == year_month
        ]

        if not filtered_scws:
            print(f"No SCWs found for {year_month}")
            continue

        # add filtering for not choosing the first 5 and last 10 ScWs of each revolution 
        # (ScWs look like RRRRPPPPSSS.001, so choose 5 < PPPP < max(PPPP) - 10)
        rrrr_max_pppp = defaultdict(int)
        for scwgr in filtered_scws:
            rrrr = scwgr[0][:4]  
            pppp = int(scwgr[0][4:8])
            rrrr_max_pppp[rrrr] = max(rrrr_max_pppp[rrrr], pppp)

        filtered_scws = [
            scwind for scwind in filtered_scws
            if 20 < int(scwind[0][4:8]) < rrrr_max_pppp[scwind[0][:4]] - 40
        ]

        if not filtered_scws:
            print(f"No SCWs found for {year_month} after filtering for pointings")
            continue
        else:
            print(f"{len(filtered_scws)} SCWs found for {year_month} after filtering for pointings")

        for scw_id, start_time, end_time, ra, dec in filtered_scws:
            print(f"Trying SCW {scw_id} for {year_month}")

            par_dict = {
                "RA": ra,
                "DEC": dec,
                "E1_keV": "15",
                "E2_keV": "30",
                "T_format": "isot",
                'T1': start_time,
                'T2': end_time,
                "detection_threshold": "5",
                "instrument": "isgri",
                "osa_version": "OSA11.2",
                "product": "isgri_image",
                "product_type": "Real",
                "scw_list": [scw_id],
                'token': disp.disable_email_token(oda_api.token.discover_token()),
            }

            if scw_id not in disp_by_date:
                disp_by_date[scw_id] = DispatcherAPI(url="https://www.astro.unige.ch/mmoda/dispatch-data", instrument="mock", wait=False)

            _disp = disp_by_date[scw_id]

            data = data_by_date.get(scw_id, None)

            if data is None and not _disp.is_failed:
                try:
                    if not _disp.is_submitted:
                        data = _disp.get_product(**par_dict, silent=True)
                    else:
                        _disp.poll()

                    if not _disp.is_complete:
                        continue  # Retry with the next SCW
                        # raise ValueError("Query incomplete")

                    # Query successful, store the data
                    data = _disp.get_product(**par_dict, silent=True)
                    data_by_date[scw_id] = data
                    image_results.append(data)
                    successful_scws.append(scw_id)
                    # break  # Stop trying other SCWs for this month (not for Jupiter)

                except Exception as e:
                    print(f"Query failed for SCW {scw_id}: {e}")
                    continue  # Try the next SCW

        else:
            print(f"All SCWs failed for {year_month}, skipping.")
            # print(f"Skipping {year_month} due to errors.")

    n_complete = len([year for year, _disp in disp_by_date.items() if _disp.is_complete])
    print(f"complete {n_complete} / {len(disp_by_date)}")

    if n_complete == len(disp_by_date):
        print("done!")
        break
    print("not done")

No SCWs found for 2003-01 after filtering for pointings
7 SCWs found for 2003-06 after filtering for pointings
Trying SCW 007700210010.001 for 2003-06
Trying SCW 007700220010.001 for 2003-06
Trying SCW 007700230010.001 for 2003-06
Query failed for SCW 007700230010.001: {'cdci_data_analysis_version': '1.3.5', 'cdci_data_analysis_version_details': 'unknown', 'config': {'dispatcher-config': {'cfg_dict': {'dispatcher': {'bind_options': {'bind_host': '0.0.0.0', 'bind_port': 8000}, 'dispatcher_callback_url_base': 'https://dispatcher-staging-flux.obsuks1.unige.ch', 'dummy_cache': 'dummy-cache', 'email_options': {'bcc_receivers_email_addresses': ['vladimir.savchenko@gmail.com'], 'cc_receivers_email_addresses': [], 'email_sending_job_submitted': True, 'email_sending_job_submitted_default_interval': 1209600, 'email_sending_timeout': True, 'email_sending_timeout_default_threshold': 1, 'sender_email_address': 'postmaster@in.odahub.io', 'smtp_port': 587, 'smtp_server': 'smtp.eu.mailgun.org'}, 'matr

KeyboardInterrupt: 

Image analysis (extraction of count rates over time)

In [ ]:
from datetime import datetime

jupiter_countrates = []
jupiter_variances = []
jupiter_annular_countrates = []
jupiter_annular_variances = []
obs_start_dates = []
obs_end_dates = []
offsets = []

for result in image_results:

    intensity_unit = result.mosaic_image_0_mosaic.get_data_unit(2)  
    header = result.mosaic_image_0_mosaic.get_data_unit(2).header
    intensity_data = intensity_unit.data 

    var_unit = result.mosaic_image_0_mosaic.get_data_unit(3)  
    var_data = var_unit.data

    timestamp = datetime.strptime(header['DATE-OBS'], "%Y-%m-%dT%H:%M:%S.%f")
    index_for_time = year_months.index(timestamp)
    current_ra = ra[index_for_time]
    current_dec = dec[index_for_time]

    wcs = WCS(header) 
    x, y = wcs.all_world2pix(ra, dec, 0)
    x_int, y_int = int(round(x.item())), int(round(y.item()))

    pointing = SkyCoord(ra=header['CRVAL1'], dec=header['CRVAL2'], unit=("deg", "deg"))
    jupiter_coords = SkyCoord(ra=current_ra, dec=current_dec, unit=("deg", "deg"))
    offset = pointing.separation(jupiter_coords).deg

    jupiter_countrates.append(intensity_data[y_int, x_int])
    jupiter_variances.append(var_data[y_int, x_int])
    obs_start_dates.append(header['DATE-OBS'])
    obs_end_dates.append(header['DATE-END'])
    offsets.append(offset)

    # Extract countrate and variance in annular region around the source
    annular_var = []
    annular_rate = []

    for x,y in range(x_int-5, x_int+5), range(y_int-5, y_int+5):
        annular_var.append(var_data[y, x])
        annular_rate.append(intensity_data[y, x])

    annular_var = np.mean(annular_var)
    annular_rate = np.mean(annular_rate)
    annular_var -= var_data[y_int, x_int] 
    annular_rate -= intensity_data[y_int, x_int]
    
    jupiter_annular_variances.append(annular_var)
    jupiter_annular_countrates.append(annular_rate)

In [ ]:
E1, E2 = 15, 30
output_filename = f"../data/jupiter_longterm_img_data_{E1}_{E2}.txt"

data = np.column_stack([np.sort(successful_scws), obs_start_dates, obs_end_dates, jupiter_countrates, jupiter_variances, offsets, jupiter_annular_countrates, jupiter_annular_variances])

header = "SCW, Obs Start Date, Obs End Date, Count Rate, Variance, Angular offset, Annular Count Rate, Annular Variance"

np.savetxt(output_filename, data, fmt="%s", delimiter=",", header=header, comments="")

print(f"Data saved to {output_filename}")

We can already do some plotting

In [ ]:
from datetime import datetime

obs_times = [datetime.strptime(date, "%Y-%m-%dT%H:%M:%S") for date in obs_start_dates]

jupiter_countrates = np.array(jupiter_countrates)
jupiter_variances = np.array(jupiter_variances)
errors = np.sqrt(jupiter_variances)
offsets = np.array(offsets)

# Calculate average and standard deviation
avg_count_rate = np.mean(jupiter_countrates)
std_count_rate = np.std(jupiter_countrates)

avg_offset = np.mean(offsets)
std_offset = np.std(offsets)

# Plot count rate over time with errorbars and std region
plt.figure()
plt.errorbar(obs_times, jupiter_countrates, yerr=errors, fmt='o', capsize=5, label='jupiter Count Rate')
plt.axhline(avg_count_rate, color='r', linestyle='--', label=f'Average Count Rate ({avg_count_rate:.2f})')
plt.fill_between(obs_times, avg_count_rate - std_count_rate, avg_count_rate + std_count_rate, color='r', alpha=0.2, label=f'Standard Deviation ({std_count_rate:.2f})')
plt.xlabel("Observation Date")
plt.ylabel("Count Rate (counts/s)")
plt.title("jupiter Count Rate Over Time")
plt.xticks(rotation=45)
plt.grid(True, linestyle="--", alpha=0.6)
plt.legend()
plt.tight_layout()

# Plot angular offset from pointing center over time with std region
plt.figure()
plt.scatter(obs_times, offsets, label='Angular Offset')
plt.axhline(avg_offset, color='g', linestyle='--', label=f'Average Offset ({avg_offset:.2f}°)')
plt.fill_between(obs_times, avg_offset - std_offset, avg_offset + std_offset, color='g', alpha=0.2, label=f'Standard Deviation ({std_offset:.2f}°)')
plt.xlabel("Observation Date")
plt.ylabel("Angular offset (degrees)")
plt.title("jupiter Angular Offset Over Time")
plt.xticks(rotation=45)
plt.grid(True, linestyle="--", alpha=0.6)
plt.legend()
plt.tight_layout()


## LIGHT CURVE

In [ ]:
api_cat={
    "cat_frame": "fk5",
    "cat_coord_units": "deg",
    "cat_column_list": [
        [0],
        ["Jupiter"],
        [125.4826889038086],
        [0], # this will be updated during the querying
        [0],
        [-32768],
        [2],
        [0],
        [0.0002800000074785203]],
    "cat_column_names": [
        "meta_ID",
        "src_names",
        "significance",
        "ra",
        "dec",
        "NEW_SOURCE",
        "ISGRI_FLAG",
        "FLAG",
        "ERR_RAD"
    ],
    "cat_column_descr":
        [
            ["meta_ID", "<i8"],
            ["src_names", "<U11"],
            ["significance", "<f8"],
            ["ra", "<f8"],
            ["dec", "<f8"],
            ["NEW_SOURCE", "<i8"],
            ["ISGRI_FLAG", "<i8"],
            ["FLAG", "<i8"],
            ["ERR_RAD", "<f8"]
        ],
    "cat_lat_name": "dec",
    "cat_lon_name": "ra"
}

In [ ]:
lc_disp_by_date = {}
lc_data_by_date = {}
successful_lc_scws = []

disp = DispatcherAPI(url="https://www.astro.unige.ch/mmoda/dispatch-data", instrument="mock")

while True:
    lc_results = []

    for year_month in set(year_months):
        filtered_scws = [
            (scw[i], isot_start_times[i], isot_end_times[i], duration_seconds[i], jupiter_ra[i], jupiter_dec[i])
            for i, year_month_in_list in enumerate(year_months)
            if year_month_in_list == year_month
        ]

        if not filtered_scws:
            print(f"No SCWs found for {year_month}")
            continue

        # add filtering for not choosing the first 5 and last 10 ScWs of each revolution 
        # (ScWs look like RRRRPPPPSSS.001, so choose 5 < PPPP < max(PPPP) - 10)
        rrrr_max_pppp = defaultdict(int)
        for scwgr in filtered_scws:
            rrrr = scwgr[0][:4]  
            pppp = int(scwgr[0][4:8])
            rrrr_max_pppp[rrrr] = max(rrrr_max_pppp[rrrr], pppp)

        filtered_scws = [
            scwind for scwind in filtered_scws
            if 5 < int(scwind[0][4:8]) < rrrr_max_pppp[scwind[0][:4]] - 10
        ]

        if not filtered_scws:
            print(f"No SCWs found for {year_month} after filtering for pointings")
            continue
        else:
            print(f"{len(filtered_scws)} SCWs found for {year_month} after filtering for pointings")

        for scw_id, start_time, end_time, duration, ra, dec in filtered_scws:
            print(f"Trying SCW {scw_id} with duration {duration} for {year_month}")

            api_cat.update({
                "cat_column_list": [
                    [0],
                    ["Jupiter"],
                    [125.4826889038086],
                    [ra],  # Update RA
                    [dec],  # Update DEC
                    [-32768],
                    [2],
                    [0],
                    [0.0002800000074785203]
                ]
            })

            par_dict = {
                "RA": ra,
                "DEC": dec,
                "E1_keV": "15",
                "E2_keV": "30", 
                "T_format": "isot",
                'T1': start_time,
                'T2': end_time,
                "time_bin": duration_seconds, 
                "instrument": "isgri",
                "osa_version": "OSA11.2",
                "product": "isgri_lc",
                "product_type": "Real",
                "scw_list": [scw_id],
                'token': disp.disable_email_token(oda_api.token.discover_token()),
                'selected_catalog': json.dumps(api_cat)
                }
            
            if scw_id not in lc_disp_by_date:
                lc_disp_by_date[scw_id] = DispatcherAPI(url="https://www.astro.unige.ch/mmoda/dispatch-data", instrument="mock", wait=False)
            
            _disp = lc_disp_by_date[scw_id]
            
            data = lc_data_by_date.get(scw_id, None)

            if data is None and not _disp.is_failed:
                try:
                    if not _disp.is_submitted:
                        data = _disp.get_product(**par_dict, silent=True)
                    else:
                        _disp.poll()

                    if not _disp.is_complete:
                        continue  # Retry with the next SCW
                        # raise ValueError("Query incomplete")  # Force skipping the month

                    # Query successful, store the data
                    data = _disp.get_product(**par_dict, silent=True)
                    lc_data_by_date[scw_id] = data
                    lc_results.append(data)
                    successful_lc_scws.append(scw_id)
                    break  # Stop trying other SCWs for this month

                except Exception as e:
                    print(f"Query failed for SCW {scw_id}: {e}")
                    continue  # Try the next SCW

        else:
            print(f"All SCWs failed for {year_month}, skipping.")
            # print(f"Skipping {year_month} due to errors.")

    n_complete = len([year for year, _disp in lc_disp_by_date.items() if _disp.is_complete])
    print(f"complete {n_complete} / {len(lc_disp_by_date)}")

    if n_complete == len(lc_disp_by_date):
        print("done!")
        break
    print("not done")

Light curve analysis

In [ ]:
from datetime import datetime

jupiter_lc_countrates = []
jupiter_lc_errors = []
lc_start_dates = []
lc_end_dates = []

for result in lc_results:

    lc = result._p_list[0]

    start_time = lc.data_unit[1].data['TSTART']
    end_time = lc.data_unit[1].data['TSTOP']
    mjd_ref = lc.data_unit[1].header['MJDREF'] 
    start_date = [Time(mjd_ref + t, format='mjd').isot for t in start_time]
    end_date = [Time(mjd_ref + t, format='mjd').isot for t in end_time]

    rate = lc.data_unit[1].data['RATE']
    error = lc.data_unit[1].data['ERROR']

    jupiter_lc_countrates.append(rate)
    jupiter_lc_errors.append(error)
    lc_start_dates.append(start_date)
    lc_end_dates.append(end_date)

Save data to file

In [ ]:
E1, E2 = 15, 30
output_lc_file = f"../data/jupiter_longterm_lc_data_{E1}_{E2}.txt"

lc_data = np.column_stack([np.sort(successful_lc_scws), lc_start_dates, lc_end_dates, jupiter_lc_countrates, jupiter_lc_errors])

lc_header = "SCW, Obs Start Date, Obs End Date, Count Rate, Error"

np.savetxt(output_lc_file, lc_data, fmt="%s", delimiter=",", header=lc_header, comments="")

print(f"Data saved to {output_lc_file}")

Plot

In [ ]:
from datetime import datetime

lc_times = [datetime.strptime(date, "%Y-%m-%dT%H:%M:%S") for date in lc_start_dates]

lc_rates = np.array(jupiter_countrates)
lc_err = np.array(jupiter_lc_errors)

plt.figure()
plt.errorbar(lc_times, lc_rates, yerr=lc_err, fmt='o', capsize=5, label="Jupiter Count Rate")
plt.xlabel("Observation Date")
plt.ylabel("Count Rate")
plt.title("Jupiter Count Rate Over Time")
plt.xticks(rotation=45)
plt.grid(True, linestyle="--", alpha=0.6)
plt.legend()
plt.tight_layout()

## COMPARISON
Here we compare the count rate of Jupiter over time between the two methods (using images or light curves).

In [ ]:
plt.figure()

plt.errorbar(obs_times, jupiter_countrates, yerr=errors, fmt='o', capsize=5, label="Images")
plt.errorbar(lc_times, lc_rates, yerr=lc_err, fmt='o', capsize=5, label="Light curves")

plt.xlabel("Observation Date")
plt.ylabel("Count Rate")
plt.title("Jupiter Count Rate Over Time")
plt.xticks(rotation=45)
plt.grid(True, linestyle="--", alpha=0.6)
plt.legend()
plt.tight_layout()